In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


MNIST 손글씨 데이터를 오토인코더를 사용해서 차원축소 후 시각화한다.

MNIST 데이터를 획득한다.

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


MNIST 데이터 전처리  
획득된 학습 데이터는 오토인코더 학습에 사용하고 테스트 데이터 중 300개의 데이터만 선택해서 시각화에 사용한다.

In [7]:
x_train = x_train.reshape(60000, 784) # 학습 데이터

# 테스트 데이터 중 300개만 선택해서 테스트 데이터로 사용한다. => 데이터 시각화에 사용한다.
x_test = x_test[:300]
y_test = y_test[:300]
x_test = x_test.reshape(300, 784)

# 데이터 정규화
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

오토인코더 구조  
텐서플로우로 구현할 오토인코더는 아래 그림과 같은 구조를 가진다.

In [11]:
from IPython.display import Image
Image('./autoEncoder.png', width='900')

텐서플로우 모델을 만든다.

In [12]:
# 텐서플로우 모델을 구현한다. 손실 함수는 입력값과 출력값의 차이를 최소화하는 것으로 설정한다.
_input = tf.placeholder(tf.float32, [None, 28 * 28])       # 입력
encoder = tf.layers.dense(_input, 128, tf.nn.tanh)         # 인코더
bottleneck = tf.layers.dense(encoder, 3)                   # 차원축소
decoder = tf.layers.dense(bottleneck, 128, tf.nn.tanh)     # 디코더
_output = tf.layers.dense(decoder, 28 * 28, tf.nn.sigmoid) # 출력
loss = tf.losses.mean_squared_error(labels=_input, predictions=_output) # 손실함수
train = tf.train.AdamOptimizer(0.002).minimize(loss)       # 최적화